In [3]:
import pandas as pd
import numpy as np 

In [4]:

# Set file path (adjust if needed)
file_path = 'data/results/trades_detailed_XAUUSD_fixed.csv'

# Read in the data
df = pd.read_csv(file_path)

# Check if 'pnl' and 'session' columns exist
if 'pnl' not in df.columns or 'session' not in df.columns:
    raise ValueError("The CSV must contain 'pnl' and 'session' columns.")

# Calculate total PnL per session
pnl_by_session = df.groupby('session')['pnl'].sum()

# Starting amount
starting_amount = 100000

# Create a result DataFrame with final balances
results = pd.DataFrame({
    'Total_PnL': pnl_by_session,
    'Final_Amount': starting_amount + pnl_by_session
})

print("PnL and final balances per session:")
print(results)


PnL and final balances per session:
           Total_PnL   Final_Amount
session                            
asian     -24.641592   99975.358408
london   7031.899911  107031.899911
us       3188.521676  103188.521676


In [9]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

# Set file path
file_path = 'data/results/trades_detailed_XAUUSD_fixed.csv'

# Read in the data
df = pd.read_csv(file_path)

# Check required columns
required_columns = {'pnl', 'session', 'exit_time'}
if not required_columns.issubset(df.columns):
    raise ValueError(f"The CSV must contain the following columns: {required_columns}")

# Convert timestamp to datetime if not already
df['timestamp'] = pd.to_datetime(df['exit_time'])

# Sort by timestamp to maintain chronological order
df = df.sort_values(by='exit_time')

# Starting amount
starting_amount = 100000

# Initialize a result dictionary
results = {}

# Risk-free rate for Sharpe ratio (adjust if needed)
risk_free_rate = 0  # Assume 0 for simplicity

# Loop through each session
for session in df['session'].unique():
    session_df = df[df['session'] == session]
    
    # Calculate cumulative returns
    cumulative_pnl = session_df['pnl'].cumsum() + starting_amount
    
    # Calculate drawdown
    running_max = cumulative_pnl.cummax()
    drawdown = (cumulative_pnl - running_max) / running_max
    max_drawdown = drawdown.min()
    
    # Calculate Sharpe ratio
    pnl_returns = session_df['pnl']
    sharpe_ratio = (pnl_returns.mean() - risk_free_rate) / pnl_returns.std(ddof=1)
    annualized_sharpe = sharpe_ratio * np.sqrt(252)

    
    # Skewness and excess kurtosis
    pnl_skewness = skew(pnl_returns)
    pnl_excess_kurtosis = kurtosis(pnl_returns)  # Already excess kurtosis
    

    
    # Store the results
    results[session] = {
        'Max_Drawdown': max_drawdown,
        'Sharpe_Ratio': sharpe_ratio,
        'Skewness': pnl_skewness,
        'Excess_Kurtosis': pnl_excess_kurtosis,
        'Annualized Sharpe': annualized_sharpe
    }

# Convert results to DataFrame for a nice display
results_df = pd.DataFrame(results).T

print("Performance metrics per session: " + file_path)
print(results_df)


Performance metrics per session: data/results/trades_detailed_XAUUSD_fixed.csv
        Max_Drawdown  Sharpe_Ratio  Skewness  Excess_Kurtosis  \
asian      -0.021978     -0.000237  0.073824        -0.882663   
london     -0.029073      0.040657  0.170889        -1.326006   
us         -0.020255      0.019587  0.251928        -0.996685   

        Annualized Sharpe  
asian           -0.003756  
london           0.645415  
us               0.310932  


In [10]:
import pandas as pd
import numpy as np
from scipy import stats

# Load your CSV (update path if needed)
df = pd.read_csv("data/results/trades_detailed_XAUUSD_fixed.csv", parse_dates=['exit_time'])

# Make sure the session column exists
assert 'session' in df.columns, "The 'session' column is missing!"

# Make sure it's sorted
df = df.sort_values('exit_time')

# Define sessions
sessions = df['session'].unique()

# Store results
results = {}

for session in sessions:
    session_df = df[df['session'] == session]

    if session_df.empty:
        print(f"No data for session: {session}")
        continue

    # Group by date (floor to day) and get the last capital_curve of the day
    capital_curve = session_df.groupby(session_df['exit_time'].dt.floor('D'))['capital_curve'].last()

    # Ensure it's continuous daily (business days), forward-fill
    capital_curve = capital_curve.asfreq('B').ffill()

    # Calculate daily returns
    daily_returns = capital_curve.pct_change().dropna()

    # Sharpe (annualized)
    mean_return = daily_returns.mean()
    std_return = daily_returns.std()
    sharpe_annualized = (mean_return / std_return) * np.sqrt(252) if std_return != 0 else np.nan

    # Skewness
    skewness = stats.skew(daily_returns)

    # Excess kurtosis
    excess_kurtosis = stats.kurtosis(daily_returns, fisher=True)

    # Max drawdown
    rolling_max = capital_curve.cummax()
    drawdowns = (capital_curve - rolling_max) / rolling_max
    max_drawdown_pct = abs(drawdowns.min()) * 100

    # Save results
    results[session] = {
        'Sharpe Ratio (annualized)': sharpe_annualized,
        'Skewness': skewness,
        'Excess Kurtosis': excess_kurtosis,
        'Max Drawdown (%)': max_drawdown_pct
    }


# Print nicely
print("\nPerformance metrics per session:")
for session, metrics in results.items():
    print(f"\n{session.upper()}:")
    print(f"Sharpe Ratio (annualized): {metrics['Sharpe Ratio (annualized)']:.4f}")
    print(f"Skewness: {metrics['Skewness']:.4f}")
    print(f"Excess Kurtosis: {metrics['Excess Kurtosis']:.4f}")
    print(f"Max Drawdown: {metrics['Max Drawdown (%)']:.2f}%")



Performance metrics per session:

ASIAN:
Sharpe Ratio (annualized): 0.0014
Skewness: -0.8343
Excess Kurtosis: 1.5044
Max Drawdown: 2.17%

LONDON:
Sharpe Ratio (annualized): 0.7763
Skewness: -1.0563
Excess Kurtosis: 0.4483
Max Drawdown: 2.91%

US:
Sharpe Ratio (annualized): 0.3654
Skewness: -1.0157
Excess Kurtosis: 0.3896
Max Drawdown: 2.03%


In [11]:
import pandas as pd
import numpy as np
from scipy import stats

# Constants
INITIAL_CAPITAL = 100_000

# Load the file
df = pd.read_csv("data/results/trades_detailed_XAUUSD_fixed.csv", parse_dates=['exit_time'])

# Make sure it's sorted
df = df.sort_values('exit_time')

# Define sessions
sessions = df['session'].unique()

# Store results
results = {}

for session in sessions:
    session_df = df[df['session'] == session].copy()

    if session_df.empty:
        print(f"No data for session: {session}")
        continue

    # Sort by exit_time
    session_df = session_df.sort_values('exit_time')

    # Rebuild the capital curve: start from INITIAL_CAPITAL
    capital_list = []
    capital = INITIAL_CAPITAL

    for _, row in session_df.iterrows():
        pnl = row['pnl']
        capital += pnl
        capital_list.append(capital)

    # Add the session-specific capital curve
    session_df['session_capital_curve'] = capital_list

    # Group by date (floor to day) to get last capital of each day
    daily_capital = session_df.groupby(session_df['exit_time'].dt.floor('D'))['session_capital_curve'].last()

    # Ensure it's continuous (business days), forward-fill
    daily_capital = daily_capital.asfreq('B').ffill()

    # Compute daily returns
    daily_returns = daily_capital.pct_change().dropna()

    # Sharpe (annualized)
    mean_return = daily_returns.mean()
    std_return = daily_returns.std()
    sharpe_annualized = (mean_return / std_return) * np.sqrt(252) if std_return != 0 else np.nan

    # Skewness
    skewness = stats.skew(daily_returns)

    # Excess kurtosis
    excess_kurtosis = stats.kurtosis(daily_returns, fisher=True)

    # Max drawdown
    rolling_max = daily_capital.cummax()
    drawdowns = (daily_capital - rolling_max) / rolling_max
    max_drawdown_pct = abs(drawdowns.min()) * 100

    # Save results
    results[session] = {
        'Sharpe Ratio (annualized)': sharpe_annualized,
        'Skewness': skewness,
        'Excess Kurtosis': excess_kurtosis,
        'Max Drawdown (%)': max_drawdown_pct
    }

# Print results nicely
print("\nSession-wise performance metrics (rebuilding per-session capital curve):")
for session, metrics in results.items():
    print(f"\n{session.upper()}:")
    print(f"Sharpe Ratio (annualized): {metrics['Sharpe Ratio (annualized)']:.4f}")
    print(f"Skewness: {metrics['Skewness']:.4f}")
    print(f"Excess Kurtosis: {metrics['Excess Kurtosis']:.4f}")
    print(f"Max Drawdown: {metrics['Max Drawdown (%)']:.2f}%")



Session-wise performance metrics (rebuilding per-session capital curve):

ASIAN:
Sharpe Ratio (annualized): 0.0014
Skewness: -0.8343
Excess Kurtosis: 1.5044
Max Drawdown: 2.17%

LONDON:
Sharpe Ratio (annualized): 0.7763
Skewness: -1.0563
Excess Kurtosis: 0.4483
Max Drawdown: 2.91%

US:
Sharpe Ratio (annualized): 0.3654
Skewness: -1.0157
Excess Kurtosis: 0.3896
Max Drawdown: 2.03%
